In [1]:
sf = {
      'account': 'cm18591.west-europe.azure',
      'user': 'LITINGAALTO',
      'password': '199341Asd'
      }
import pandas as pd
from snowflake.sqlalchemy import URL
from sklearn.metrics.pairwise import cosine_similarity
from sqlalchemy import create_engine
import seaborn as sns
import numpy as np

engine = create_engine(URL(
            account = sf['account'],
            user = sf['user'],
            password = sf['password'],
            role = 'SYSADMIN',
            database = 'SEURE_DM_DEV',
            warehouse = 'ANALYTICS_WH',
            schema = 'ANALYTIIKKA',
            numpy = True
        ))
import warnings
warnings.filterwarnings('ignore')

In [2]:
from datetime import date
from dateutil.relativedelta import relativedelta
import requests 
import json 
import math
today=date.today()
toimialaid=[ 10, 20, 30, 40, 60, 70, 90, 100]
# "10 Terveydenhuolto, 20 Varhaiskasvatus, 30 Opetus, 40 Sosiaali, 60 Keittiö ja siivous, 70 Kiinteistö ja valvonta,  90 Hallinto- ja toimisto, 100 Muut"

query=f"select * from VIEW_TOTEUMA_CONFIRM2 where toimiala={toimialaid[0]}"
query1=f"select tyontekijaid, erityisominaisuusid from staging_test.public.STG_KEIKKANETTI_tblTyontekijaErityisominaisuudet\
 where viimeinenvoimassaolopaiva is null or  viimeinenvoimassaolopaiva >='{today}'"  
query2=f"select * from VIEW_TYO_FEATURE2 where toimiala = {toimialaid[0]}"
query4 ='select osoite, postinumero, postitoimipaikka, tyontekija_id from "SEURE_DM_PROD"."LIIKETOIMINTA_HENKILOTASO"."VIEW_D_TYONTEKIJA"  where KEIKKAILUKELPOINEN = True and (osoite is not null and postinumero is not null)'
query5=f"select distinct asiakasid, tyontekijaid from staging_test.public.STG_KEIKKANETTI_tblTyontekijaSulkulista where tyyppi =2 and \
asiakasid in (select distinct id from  staging_test.public.STG_KEIKKANETTI_tblAsiakas where kustannuspaikkaID = {toimialaid[0]})"
query6='select distinct  tyontekijaid,asiakasid from staging_test.public.STG_KEIKKANETTI_tblTyontekijaSulkulista where tyyppi =1'
query7 = "select * from view_varattu_tyontekjia_today"
query8 = "select * from view_tyontekija_ammattinimike"
query9 = f"select * from view_tilaus_lisakriteeri2 where toimiala_id = '{toimialaid[0]}'"

connection = engine.connect()
df = pd.read_sql(query, connection)
df_employee=pd.read_sql(query1, connection)
df_job=pd.read_sql(query2, connection)
emp_add = pd.read_sql(query4, connection)
asiakas_sulkulista = pd.read_sql(query5, connection)
tyontekija_sulkulista = pd.read_sql(query6, connection)
emp_varattu= pd.read_sql(query7, connection)
emp_amma=pd.read_sql(query8, connection)
tilaus_erityisominaisuus= pd.read_sql(query9, connection)
emp_amma.nimi = emp_amma.nimi.apply(lambda x: x.strip())
tilaus_erityisominaisuus = tilaus_erityisominaisuus.fillna(0)
# convert 258 to 351
df_employee.loc[df_employee.erityisominaisuusid==258, 'erityisominaisuusid']=351

### preprocessing

In [3]:
import math
df['nimi']=df['nimi'].apply(lambda x: str(x).strip())
df2019=df[['tyontekijaid','asiakasid','maara','nimi','asiakasnimi','tilausid']]
df2019['job']=df2019['asiakasid'].astype(str)+";"+df2019['nimi']

df2019=df2019.groupby(['tyontekijaid','job'])['maara'].sum().reset_index()
df2019.columns=['tyontekija_id','job','tyovuorot_taytetyt']
df2019.tyontekija_id=df2019.tyontekija_id.astype(int)
df2019 = df2019[df2019['tyovuorot_taytetyt']>0]
df2019['tyovuorot_taytetyt']=df2019['tyovuorot_taytetyt'].apply(lambda x: math.ceil(x/7.5))
df2019['tyovuorot_taytetyt']=df2019['tyovuorot_taytetyt'].astype(int)
n_employees = df2019.tyontekija_id.unique().shape[0]
n_jobs = df2019.job.unique().shape[0]
print("unique employees: {}, unique_jobs: {}".format(n_employees, n_jobs))
df2019.sort_values('tyovuorot_taytetyt', ascending = False).head(3)

unique employees: 5067, unique_jobs: 1631


,tyontekija_id,job,tyovuorot_taytetyt
3206,33598,"7533;Lähihoitaja, vuodeosasto",127
16925,129666,"18973;Lähihoitaja, kotihoito",123
13395,111459,"13776;Lähihoitaja, kotihoito lääkehoitoluvaton",121


In [4]:
# normalize the 5% outliers 
a=df2019.groupby(['tyontekija_id','job']).sum().reset_index().sort_values('tyovuorot_taytetyt',ascending = False)
need_scale=a[a['tyovuorot_taytetyt']>a['tyovuorot_taytetyt'].quantile(0.95)]
rest = a[a['tyovuorot_taytetyt']<=a['tyovuorot_taytetyt'].quantile(0.95)]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(a['tyovuorot_taytetyt'].quantile(0.95),a['tyovuorot_taytetyt'].quantile(0.95)+8))
scaler.fit(need_scale['tyovuorot_taytetyt'].values.reshape(-1,1))
need_scale['tyovuorot_taytetyt']=scaler.transform(need_scale['tyovuorot_taytetyt'].values.reshape(-1,1))
# append the data with the modified 5%
df_scaled=rest.append(need_scale)
df_scaled['jobs']=df_scaled['job'].apply(lambda x: str(x).strip())
df_scaled['job']=df_scaled['jobs']
ratings=df_scaled.pivot_table(index='tyontekija_id', columns='job', values='tyovuorot_taytetyt').fillna(0)

### Feature data handling

In [5]:
# This is the jobs that have orders in past 6kk, terveydenhuolto
df_terv=df_job
df_terv['job']=df_terv['job'].apply(lambda x: str(x).strip())
df['job']=df['asiakasid'].astype(str)+";"+df['nimi']

terv2019=pd.DataFrame(df.groupby(['tyontekijaid','job'])['maara'].sum()).reset_index()
df_terv=terv2019.merge(df_terv[['job', 'katuosoite','postinumero','postitoimipaikka','erityisominaisuusid']], how='left', on='job')
a=df_terv[['job', 'erityisominaisuusid']]
a.fillna(0, inplace=True)
a['arvo']=1
tyo_taidot=a.pivot_table(index='job', columns='erityisominaisuusid').fillna(0)
tyo_taidot=tyo_taidot['arvo'].drop(0,1)
print('there are {} jobs and {} erityisominaisuusid in tyo_taidot'.format(tyo_taidot.shape[0], tyo_taidot.shape[1]))

there are 1632 jobs and 61 erityisominaisuusid in tyo_taidot


In [6]:
b=df_employee[['tyontekijaid','erityisominaisuusid']]
b['arvo']=1
x=b.pivot_table(index='tyontekijaid', columns='erityisominaisuusid').fillna(0)['arvo']
c=list(set(tyo_taidot.columns) & set(x.columns))
if len(list(set(tyo_taidot.columns)-set(c))) != 0:
    print('None of the employees have {} skills as required in jobs'.format(list(set(tyo_taidot.columns)-set(c))))
tyo_taidot=tyo_taidot[c]
employee_taidot=x[c]
#These are the employees that are missing erityisominaisuus
bb=set(df.tyontekijaid.unique())- set(employee_taidot.index.tolist())
print('Employee(s) that do not have any features: {}'.format(bb))
# Add the missing employess to this employee matrix
employee_taidot=employee_taidot.append(pd.DataFrame(np.zeros((len(bb),employee_taidot.shape[1])), index=list(set(df.tyontekijaid.unique())- set(employee_taidot.index.tolist())), columns=c))
print('there are {} employees and {} erityisominaisuusid in employee feature table'.format(x.shape[0], x.shape[1]))
print('filtering only common erityisominaisuusid with job+add missing employees...\nthere are {} employees and {} common erityisominaisuusid in employee feature table'.format(employee_taidot.shape[0], employee_taidot.shape[1]))

Employee(s) that do not have any features: set()
there are 54332 employees and 438 erityisominaisuusid in employee feature table
filtering only common erityisominaisuusid with job+add missing employees...
there are 54332 employees and 61 common erityisominaisuusid in employee feature table


### Load coordinates for employees and customers & sulkulista

In [7]:
xs=pd.read_excel('employee_cod.xlsx')
xs=xs.drop('Unnamed: 0',1)
xs.columns=['tyontekijaid','address','lat','lon']
coordinate_dictionary={}
for _,i in xs.iterrows():
    coordinate_dictionary[i.tyontekijaid]= [i.address, i.lat, i.lon]
employee_cod=pd.DataFrame.from_dict(coordinate_dictionary, orient='index')
employee_cod.columns=['address','lat','lon']
a=pd.read_excel('customer_cod.xlsx')
customer_cod=a[['ASIAKAS_ID','katunimi_tulos','x','y' ]].set_index('ASIAKAS_ID')
customer_cod.columns=['address','lat','lon']
# Make customer dictionary
coordinate_dictionary_customer={}
for _,i in pd.read_excel('customer_cod.xlsx').iterrows():
    coordinate_dictionary_customer[i.ASIAKAS_ID]= [i.x, i.y]

In [8]:
asiakas_sulkulista['arvo']=1
asiakas_sulkulista=asiakas_sulkulista.pivot_table(index='asiakasid', columns='tyontekijaid').fillna(0)['arvo']
tyontekija_sulkulista['arvo']=1
tyontekija_sulkulista=tyontekija_sulkulista.pivot_table(index='tyontekijaid', columns='asiakasid').fillna(0)['arvo']

### First Model: Adhoc job recommendation

In [10]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.linalg import svds
from math import sin, cos, sqrt, atan2, radians
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import statistics
import math
import itertools

class adhoc:
    def __init__(self,coordinate_dictionary,coordinate_dictionary_customer,tyo_taidot,employee_taidot, emp_amma, emp_varattu, emp_add):
        self.__coordinate_dictionary=coordinate_dictionary
        self.__coordinate_dictionary_customer = coordinate_dictionary_customer
        self.__tyo_taidot=tyo_taidot
        self.__employee_taidot=employee_taidot
        self.__emp_amma = emp_amma
        self.__emp_varattu = emp_varattu
        self.__emp_add = emp_add
        
    def calculate_station_distance_c(self,customerid, lon2, lat2 ):
        """
        :customerid: customerid 
        :lon2: longnitude of employees
        :lat2: lagnitude of employees
        return: calculated distance between customer and employees
        """
        lon1 = radians(self.__coordinate_dictionary_customer[customerid][1])
        lat1 = radians(self.__coordinate_dictionary_customer[customerid][0])
        a = math.sin(( lat1- lat2) / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin((lon1-lon2) / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = 6373 * c
        return distance
        
    def validate_employee(self,job):
        """
        :job: input job (a job is [customerid;ammattinimike])
        return validated recommendation in dataframe format with columns: 'tyontekijaid','score','threshold'
        * if employee address or customer address missing, then return empty list
        """
        amma = job.split(';')[1].strip()
        eligible=self.__emp_amma[self.__emp_amma['nimi']==amma].tyontekijaid.unique().tolist()
        emp_new = self.__employee_taidot[self.__employee_taidot.index.isin(eligible)]
        emp_new = emp_new[~emp_new.index.isin(self.__emp_varattu.tyontekija_id.tolist())]
        if job in self.__tyo_taidot.index:
            xx=self.__tyo_taidot.loc[job,].values.reshape(-1,1)
            a=emp_new.dot(xx).reset_index()
            a['threshold']=self.__tyo_taidot.loc[job,].sum()
            a=a[a[0]>=a['threshold']]
            a.columns=['tyontekijaid','score','threshold']
            return a
        else:
            a=pd.DataFrame(emp_new.index.tolist())
            a.columns=['tyontekijaid']
            a=a[a.tyontekijaid.isin(self.__emp_add.tyontekija_id.unique().tolist())]
            if a.shape[0] ==0:
                return []
            else:
                return a
    
    
    def eligible_employee_dict(self,job,k=10, km=10):
         """
        :job: input job (a job is [customerid;ammattinimike])
        :k: number of recommendations shown, default is 10
        :km: maximum distance between employees and job, default is 10
        return recommended employees with their corresponding distance to the job
        """
        good_distance=[]
        try:
            for i in self.validate_employee(job).tyontekijaid.tolist():
                if int(i) in self.__coordinate_dictionary.keys():
                    lat2, lon2= radians(self.__coordinate_dictionary[int(i)][1]), radians(self.__coordinate_dictionary[int(i)][2])
                    distance = self.calculate_station_distance_c( int(job.split(';')[0]), lon2, lat2 )
                    distance = round(distance, 1)
                    if distance <= km:
                        good_distance.append([i, distance])
        except:
            print('customer address unknown')
            if (isinstance(self.validate_employee(job), pd.DataFrame)):
                for i in self.validate_employee(job).tyontekijaid.tolist():
                    good_distance.append([i, -1])
            else:
                print('no eligible employees')
            

        good_distance = sorted(good_distance, key=lambda x: x[1])
        try:
            for i in good_distance[:k]:
                print(i)
            return good_distance[:k]
        except:
            for i in good_distance:
                print(i)
            return good_distance



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 67)

In [ ]:
c=adhoc(coordinate_dictionary,coordinate_dictionary_customer,tyo_taidot,employee_taidot, emp_amma, emp_varattu, emp_add)
c.eligible_employee_dict('18842;Lähihoitaja, varahenkilöstö')


### Second model: Job_recommendation with tilausID

In [ ]:
# Preprocessing for job_recommendation with tilaus id
# As need to consider also the "erityisominaisuusid" per order, therefore need below aggregations
df_job['job']=df_job['job'].apply(lambda x: str(x).strip())
tilaus_erityisominaisuus['job']=tilaus_erityisominaisuus['job'].apply(lambda x: str(x).strip())
tyo_taidot1=df_job[['job','erityisominaisuusid']].drop_duplicates()
b=df_employee[['tyontekijaid','erityisominaisuusid']]
b['arvo']=1
employee_taidot1=b.pivot_table(index='tyontekijaid', columns='erityisominaisuusid').fillna(0)['arvo']
tilaus_taidot1=tilaus_erityisominaisuus[['lyhyt_tilaus_id','lisakriteeri_id','job']].drop_duplicates()

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.linalg import svds
from math import sin, cos, sqrt, atan2, radians
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import statistics
import math
import itertools

class job_recommendation:
      
    def __init__(self,ratings, employee_taidot1,tilaus_taidot1, tyo_taidot1,coordinate_dictionary,
                 coordinate_dictionary_customer,asiakas_sulkulista, emp_amma, emp_varattu, emp_add):
        self.__ratings=ratings.fillna(0)
        self.__employee_taidot=employee_taidot1
        self.__tyo_taidot=tyo_taidot1
        self.__tilaus_taidot=tilaus_taidot1
        self.__coordinate_dictionary=coordinate_dictionary
        self.__coordinate_dictionary_customer=coordinate_dictionary_customer
        self.__asiakas_sulkulista=asiakas_sulkulista
        self.__emp_amma = emp_amma
        self.__emp_varattu = emp_varattu
        self.__emp_add = emp_add   

    def get_recommendations(self,tilausid, k=30):
        """
        :tilausid: orderID
        :k: number of recommendation
        return: recommended employees in dataframe format
        """
        job=self.__tilaus_taidot[self.__tilaus_taidot.lyhyt_tilaus_id==tilausid].job.unique()[0]
        try:
            idx = [i[0] for i in list(enumerate(self.__ratings.columns)) if i[1]==job][0]
            cosine_sim = linear_kernel(self.__ratings.T.values, self.__ratings.T.values)
            sim_scores = list(enumerate(cosine_sim[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            job_indices = [i[0] for i  in sim_scores[1:k] if i[1]>0]
            sim_sum=sum([i[1] for i in sim_scores[1:k] if i[1]>0])
            sim_norm=[i[1]/sim_sum for i in sim_scores[1:k] if i[1]>0]
            if len(job_indices)==0:
                a=self.__ratings[job][self.__ratings[job]>0].sort_values(ascending=False)
                a=pd.DataFrame(a).reset_index()
                a.columns=['tyontekijaid','shifts']
                return a
            else:
                x=0
                for j,s in zip(job_indices, sim_norm):
                    if len(set(self.__ratings[job][self.__ratings[job]>0].index) & set(self.__ratings.iloc[:,j][self.__ratings.iloc[:,j]>0].index)) >3:
                        x+=self.__ratings.iloc[:,j]*s
                x_i=(x*0.5+ self.__ratings[job]*0.5).sort_values(ascending=False)
                x_i=pd.DataFrame(x_i[x_i>0][:k]).reset_index()
                x_i.columns=['tyontekijaid','shifts']
                return x_i
        except:
            print('This job did not exist in past 6kk')
            return []
        
    def svd(self):
        """
        return: job-employee matrix with SVD aggregation
        """
        R = self.__ratings.values
        user_ratings_mean = np.mean(R, axis = 1)
        R_demeaned = R - user_ratings_mean.reshape(-1, 1)
        U, sigma, Vt = svds(R_demeaned, k = 50)
        sigma = np.diag(sigma)

        all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
        preds_df = pd.DataFrame(all_user_predicted_ratings, columns = self.__ratings.columns, index=self.__ratings.index)
        a=pd.DataFrame(preds_df.values.astype('int'),columns=preds_df.columns, index=preds_df.index)
        return a
   
    def svd_rec_job(self,tilausid,k=30):
        """
        :tilausid: orderID
        :k: number of recommendation, default is 30
        return: recommended employees in dataframe format
        """
        job=self.__tilaus_taidot[self.__tilaus_taidot.lyhyt_tilaus_id==tilausid].job.unique()[0]
        preds_df=self.svd()
        try:
            a=pd.DataFrame(preds_df[job].sort_values(ascending=False)[:k]).reset_index()
            if a[a[job]>0.1].shape[0]>0:
                a=a[a[job]>0.5]
                a.columns=['tyontekijaid','shifts']
                return a
            else:
                a=self.__ratings[job][self.__ratings[job]>0].sort_values(ascending=False)[:k]
                a=pd.DataFrame(a).reset_index()
                a.columns=['tyontekijaid','shifts']
                return a
        except:
            return []
            pass
 
    def validate_employee(self, tilausid):
        """
        :tilausid: orderID
        return: validated recommendation from previous functions (get_recommendation & svd_rec_job) in dataframe format
        """
        job=self.__tilaus_taidot[self.__tilaus_taidot.lyhyt_tilaus_id==tilausid].job.unique()[0]
        amma = job.split(';')[1].strip()
        eligible=self.__emp_amma[self.__emp_amma['nimi']==amma].tyontekijaid.unique().tolist()
        emp_new = self.__employee_taidot[self.__employee_taidot.index.isin(eligible)] 
        emp_new = emp_new[~emp_new.index.isin(self.__emp_varattu.tyontekija_id.tolist())]
        
        co_job=self.__tilaus_taidot[self.__tilaus_taidot.lyhyt_tilaus_id==tilausid].job.unique()[0]
        aa=set(self.__tilaus_taidot[self.__tilaus_taidot.lyhyt_tilaus_id==tilausid].lisakriteeri_id)
        bb=set(self.__tyo_taidot[self.__tyo_taidot.job==co_job].erityisominaisuusid)
        ax=aa|bb
        combined_l=list(ax-set([0]))
        try:
            eligible_employee_list = list(emp_new.loc[emp_new[combined_l].sum(1)==len(combined_l)].index)
            a=pd.DataFrame(eligible_employee_list)
            a.columns=['tyontekijaid']
            
            return a
        except:
            
            print('there are requirements that none employees have')
            return []
    
    def calculate_station_distance_c(self,customerid, lon2, lat2 ):
        """
        :customerid: customerID
        :lon2: longnitude (of each employee)
        :lat2: lagnitude (of each employee)
        return: distance between employee and customers
        """
        lon1 = radians(self.__coordinate_dictionary_customer[customerid][1])
        lat1 = radians(self.__coordinate_dictionary_customer[customerid][0])
        a = math.sin(( lat1- lat2) / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin((lon1-lon2) / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = 6373 * c
        return distance

    def content_based_c(self,tilausid,km=10):
        """
        :tilausid: orderID
        :km: maximum distance between job and employees (default is 10)
        return: recommendation within the defined distance in dataframe format 
        """
        ok=[]
        job=self.__tilaus_taidot[self.__tilaus_taidot.lyhyt_tilaus_id==tilausid].job.unique()[0]
        amma = job.split(';')[1].strip()
        eligible=self.__emp_amma[self.__emp_amma['nimi']==amma].tyontekijaid.unique().tolist()
        emp_new = self.__employee_taidot[self.__employee_taidot.index.isin(eligible)] 
        emp_new = emp_new[~emp_new.index.isin(self.__emp_varattu.tyontekija_id.tolist())]
        
        co_job=self.__tilaus_taidot[self.__tilaus_taidot.lyhyt_tilaus_id==tilausid].job.unique()[0]
        aa=set(self.__tilaus_taidot[self.__tilaus_taidot.lyhyt_tilaus_id==tilausid].lisakriteeri_id)
        bb=set(self.__tyo_taidot[self.__tyo_taidot.job==co_job].erityisominaisuusid)
        ax=aa|bb
        combined_l=list(ax-set([0]))
        if len(combined_l)>0:
            eligible_employee_list = list(emp_new.loc[emp_new[combined_l].sum(1)==len(combined_l)].index)
            a=pd.DataFrame(eligible_employee_list)
            a.columns=['tyontekijaid']
            try:               
                for _, i in a.iterrows():
                    if int(i.tyontekijaid) in self.__coordinate_dictionary.keys():
                        lat2, lon2= radians(self.__coordinate_dictionary[int(i.tyontekijaid)][1]), radians(self.__coordinate_dictionary[int(i.tyontekijaid)][2])
                        distance = self.calculate_station_distance_c( int(job.split(';')[0]), lon2, lat2 )
                        #need to debug in case customer address is not found
                        if distance <=km:
                            ok.append(i.tyontekijaid)
                if len(ok) != 0:
                    return a[a.tyontekijaid.isin(ok)]
                else:
                    ac=list(a.tyontekijaid)
                    a4=self.__emp_add[self.__emp_add.tyontekija_id.isin(ac)].tyontekija_id.tolist()
                    if len(a4)!=0:
                        a4=pd.DataFrame(a4)
                        a4.columns=['tyontekijaid']
                        return a4
                    elif len(a4) ==0:
                        return []
            except:
                ac=list(a.tyontekijaid)
                a3=self.__emp_add[self.__emp_add.tyontekija_id.isin(ac)].tyontekija_id.tolist()
                if len(a3)!=0:
                    a3=pd.DataFrame(a3)
                    a3.columns=['tyontekijaid']
                    return a3
                elif len(a3) ==0:
                    return []

        elif len(combined_l) == 0:
            a=pd.DataFrame(emp_new.index)
            a.columns=['tyontekijaid']
            try:
                for _, i in a.iterrows():
                    if int(i.tyontekijaid) in coordinate_dictionary.keys():
                        lat2, lon2= radians(coordinate_dictionary[int(i.tyontekijaid)][1]), radians(coordinate_dictionary[int(i.tyontekijaid)][2])
                        distance = calculate_station_distance_c( int(job.split(';')[0]), lon2, lat2 )
                        if distance <=km:
                            ok.append(i.tyontekijaid)
                if len(ok) != 0:
                    return a[a.tyontekijaid.isin(ok)]
                else:
                    ac=list(a.tyontekijaid)
                    a1=self.__emp_add[self.__emp_add.tyontekija_id.isin(ac)].tyontekija_id.tolist()
                    if len(a1)!=0:
                        a1=pd.DataFrame(a1)
                        a1.columns=['tyontekijaid']
                        return a1
                    elif len(a1) ==0:
                        return []
                    
            except:
                ac=list(a.tyontekijaid)
                a2=self.__emp_add[self.__emp_add.tyontekija_id.isin(ac)].tyontekija_id.tolist()
                if len(a2)!=0:
                    a2=pd.DataFrame(a2)
                    a2.columns=['tyontekijaid']
                    return a2
                elif len(a2) ==0:
                    return []
            
        
    def hybrid_recommendation_employee_w_sulku(self,tilausid, k=20, km=10):
        """
        :tilausid: orderID
        :k: number of recommendation shown (default 20)
        :km: max distance between employees and job (default 10)
        return: recommendation ordered first by num of erityisominaisuudet, then by the predicted shifts of an employee
        """
        sim=self.get_recommendations(tilausid)
        svd=self.svd_rec_job(tilausid)
        if isinstance(svd, pd.DataFrame):
            svd_emp=svd.tyontekijaid.unique().tolist()
        else:
            svd_emp=[]
        if isinstance(sim, pd.DataFrame):
            sim_emp=sim.tyontekijaid.unique().tolist()
        else:
            sim_emp =[]
        common_emp=(set(svd_emp) & set(sim_emp))
        content=self.content_based_c(tilausid,km)         
        if (isinstance(svd, pd.DataFrame)) | (isinstance(sim, pd.DataFrame)):

            eligible_common_emp = [i for i in common_emp if i in self.validate_employee(tilausid).tyontekijaid.tolist()]
            svd_others=[i for i in list(set(svd_emp) - set(common_emp)) if i in self.validate_employee(tilausid).tyontekijaid.tolist()]
            sim_others=[i for i in list(set(sim_emp) - set(svd_emp)) if i in self.validate_employee(tilausid).tyontekijaid.tolist()]

            b=svd[svd.tyontekijaid.isin(svd_others)].append(sim[sim.tyontekijaid.isin(sim_others)]).sort_values('shifts', ascending=False)

            job=self.__tilaus_taidot[self.__tilaus_taidot.lyhyt_tilaus_id==tilausid].job.unique()[0]
            jobid=int(job.split(';')[0])
            if jobid not in self.__asiakas_sulkulista.index.tolist():
                cf_combined=svd[svd.tyontekijaid.isin(eligible_common_emp)].append(b)
                if cf_combined.shape[0]<k:
                    combined=pd.DataFrame(cf_combined).append(pd.DataFrame(content[:int(k-cf_combined.shape[0])].tyontekijaid)).reset_index().drop('index',1).fillna(0)
                    print(combined.tyontekijaid.reset_index().drop('index',1))
                    return combined.tyontekijaid.reset_index().drop('index',1)
                else:
                    print(combined)
                    print('a1')
                    print(cf_combined.tyontekijaid[:k].reset_index().drop('index',1))
                    return cf_combined.tyontekijaid[:k].reset_index().drop('index',1)
            else:
                searchfor=[i for i in self.__asiakas_sulkulista.loc[jobid][self.__asiakas_sulkulista.loc[jobid]>0].index.tolist()]
                cf_combined=svd[svd.tyontekijaid.isin(eligible_common_emp)].append(b)
                cf_combined=cf_combined[~cf_combined.tyontekijaid.isin(searchfor)]
                if cf_combined.shape[0]<k:
                    combined=pd.DataFrame(cf_combined).append(pd.DataFrame(content[:int(k-cf_combined.shape[0])].tyontekijaid)).reset_index().drop('index',1).fillna(0)
                    combined=combined[~combined.tyontekijaid.isin(searchfor)]
                    print(combined.tyontekijaid.reset_index().drop('index',1))
                    return combined.tyontekijaid.reset_index().drop('index',1)
                else:
                    print(cf_combined.tyontekijaid[:k].reset_index().drop('index',1))
                    return cf_combined.tyontekijaid[:k].reset_index().drop('index',1)
        elif isinstance(content, pd.DataFrame):
            job=self.__tilaus_taidot[self.__tilaus_taidot.lyhyt_tilaus_id==tilausid].job.unique()[0]
            jobid=int(job.split(';')[0])
            if jobid not in self.__asiakas_sulkulista.index.tolist():
                cf_combined=content
                if cf_combined.shape[0]<k:
                    print(cf_combined.tyontekijaid.reset_index().drop('index',1))
                    return cf_combined.reset_index().drop('index',1)
                else:
                    print(cf_combined.tyontekijaid[:k].reset_index().drop('index',1))
                    return cf_combined[:k].reset_index().drop('index',1)
            else:
                searchfor=[i for i in self.__asiakas_sulkulista.loc[jobid][self.__asiakas_sulkulista.loc[jobid]>0].index.tolist()]
                cf_combined=content
                cf_combined=cf_combined[~cf_combined.tyontekijaid.isin(searchfor)]
                if cf_combined.shape[0]<k:
                    print(cf_combined.tyontekijaid.reset_index().drop('index',1))
                    return cf_combined.reset_index().drop('index',1)
                else:
                    print(cf_combined.tyontekijaid[:k].reset_index().drop('index',1))
                    return cf_combined[:k].reset_index().drop('index',1)

        else:
            return []

In [ ]:
# Check here what are the current orderID, currently this recommendation model take the order ID that are present or in 
# future (not in past)
tilaus_erityisominaisuus

In [ ]:
b=job_recommendation(ratings, employee_taidot1,tilaus_taidot1, tyo_taidot1,coordinate_dictionary,
                 coordinate_dictionary_customer,asiakas_sulkulista, emp_amma, emp_varattu, emp_add)
b.hybrid_recommendation_employee_w_sulku(13241629)